<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [2]:
!pip install gdown

  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - done
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Created wheel for gdown: filename=gdown-3.13.0-py3-none-any.whl size=9034 sha256=162aced63bd1a573acee54386f00b0327bd8104aeb1b7239344c0b7154bd6db2
  Stored in directory: /home/jianhui/.cache/pip/wheels/04/51/53/ed3e97af28b242e9eb81afb4836273fbe233a14228aa82fea3
Successfully built gdown


In [3]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [4]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [5]:
# download and extract model files
os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
os.system('unzip {}'.format(folder_name))
!ls

checkpoint_0040.pth.tar
config.yml
events.out.tfevents.1610927742.4cb2c837708d.2694093.0
mini_batch_logistic_regression_evaluator.ipynb
resnet50_50-epochs_stl10.zip
training.log


In [6]:
!pwd

/home/jianhui/MSO/SimCLR/feature_eval


In [2]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print("Using device:", device)

Using device: cpu


In [4]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_MSO_data_loaders(shuffle=False, batch_size=256):
  from datasets.mso_dataset import MSODataset

  dataset = MSODataset(root_dir='../../MSO/Preliminary_Image_Dataset',
                        transform=transforms.Compose(
                            [transforms.Resize((224,224)),
                             transforms.ToTensor()]
                        ))

  train_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_size
  train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)

  return train_loader, test_loader

In [5]:
with open(os.path.join('./config.yml')) as file:
  config = yaml.safe_load(file)

In [6]:
config

{'arch': 'resnet18',
 'batch_size': 128,
 'data': '../datasets',
 'dataset_name': 'MSO',
 'disable_cuda': False,
 'epochs': 40,
 'fp16_precision': False,
 'gpu_index': 0,
 'log_every_n_steps': 100,
 'lr': 0.0003,
 'n_views': 2,
 'out_dim': 128,
 'seed': None,
 'temperature': 0.07,
 'weight_decay': 0.0001,
 'workers': 12}

In [7]:
config['arch']

'resnet18'

In [8]:
if config['arch'] == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=11).to(device)
elif config['arch'] == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [9]:
checkpoint = torch.load('checkpoint_0200.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [10]:
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [11]:
if config['dataset_name'] == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config['dataset_name'] == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
elif config['dataset_name'] == 'MSO':
  train_loader, test_loader = get_MSO_data_loaders()
print("Dataset:", config['dataset_name'])

Dataset: MSO


In [12]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [14]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [15]:
epochs = 100
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")


/home/jianhui/anaconda3/envs/SimCLR/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0	Top1 Train accuracy 15.408587455749512	Top1 Test accuracy: 24.8147029876709	Top5 test acc: 72.16546630859375
Epoch 1	Top1 Train accuracy 36.921348571777344	Top1 Test accuracy: 40.99392318725586	Top5 test acc: 87.50667572021484
Epoch 2	Top1 Train accuracy 50.74835968017578	Top1 Test accuracy: 50.97990036010742	Top5 test acc: 91.76348876953125
Epoch 3	Top1 Train accuracy 56.63116455078125	Top1 Test accuracy: 53.453857421875	Top5 test acc: 93.07391357421875
Epoch 4	Top1 Train accuracy 60.49281311035156	Top1 Test accuracy: 55.406982421875	Top5 test acc: 93.26922607421875
Epoch 5	Top1 Train accuracy 62.36994552612305	Top1 Test accuracy: 59.20806884765625	Top5 test acc: 93.65985107421875
Epoch 6	Top1 Train accuracy 64.22454833984375	Top1 Test accuracy: 59.92421340942383	Top5 test acc: 93.85516357421875
Epoch 7	Top1 Train accuracy 65.42736053466797	Top1 Test accuracy: 62.0893440246582	Top5 test acc: 94.1806869506836
Epoch 8	Top1 Train accuracy 66.33851623535156	Top1 Test accuracy: 62.

In [17]:
torch.save(model.state_dict(), 'fine_tuned_MSO_prelim.pth')